In [11]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd

In [12]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [13]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [14]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [15]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [16]:
df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="events", properties=jdbc_properties))

In [17]:
df_silver = df.select("silver")
df_bronze = df.select("bronze")

In [18]:
df_unionall = df_silver.unionAll(df_bronze)

In [19]:
df = df.withColumnRenamed("silver", "silver_df")
join_condition = df["gold"] == df_unionall["silver"]
join_df = df.join(df_unionall,join_condition, 'left').where("silver is null")
agg_df = join_df.groupby("gold").agg(expr("count(*) as cnt")).show()

+----------------+---+
|            gold|cnt|
+----------------+---+
|          Ronald|  1|
|         Charles|  3|
|Amthhew Mcgarray|  1|
|         jessica|  1|
|          Thomas|  3|
+----------------+---+



In [20]:
spark.stop()